In [9]:
import tabata as tbt
from tabata import *

In [10]:
storename = "../tabata/notebooks/data/in/AFL1EB.h5"
ds = Opset(storename)

In [13]:
import pandas as pd
import numpy as np

def clean_data(input_file, output_file):
    # Chargement des données
    ds = Opset(input_file)
    
    # Création d'un nouvel Opset pour le fichier propre
    clean_ds = Opset(output_file)
    clean_ds.clean()  # Nettoie le fichier de destination s'il existe déjà
    
    # Parcours des enregistrements
    for df in ds.iterator():
        if max(df["F[N]"]) > 0:  # On vérifie si les données sont valides
            x = df.index
            t = (x - x[0]).total_seconds()
            dt = np.diff(t)
            i = np.argwhere(dt != dt[1])  # Détecter un problème dans l'horodatage
            if len(i) > 0:
                # Correction de l'horodatage
                name = df.index.name
                df.index = pd.date_range(x[0], periods=len(df), freq=x[1] - x[0])  # Recalcule les timestamps
                df.index.name = name  # Récupère le nom de l'index
                
            # Enregistrement des données propres
            clean_ds.put(df)
    
    print(f"Fichier nettoyé sauvegardé sous {output_file}")

input_file = "../tabata/notebooks/data/in/AFL1EB.h5"
output_file = "../Data/cleaned/AFL1EB_cleaned.h5"
clean_data(input_file, output_file)


C:\Users\ruber\OneDrive\Bureau\Projet\tabata\opset.py:256: FutureWarning:

Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.

C:\Users\ruber\OneDrive\Bureau\Projet\tabata\opset.py:256: FutureWarning:

Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.

C:\Users\ruber\OneDrive\Bureau\Projet\tabata\opset.py:256: FutureWarning:

Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.

C:\Users\ruber\OneDrive\Bureau\Projet\tabata\opset.py:256: FutureWarning:

Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.

C:\Users\ruber\OneDrive\Bureau\Projet\tabata\opset.py:256: FutureWarning:

Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.

C:\Users\ruber\OneDrive\Bureau\Proj

Fichier nettoyé sauvegardé sous ../Data/cleaned/AFL1EB_cleaned.h5


C:\Users\ruber\OneDrive\Bureau\Projet\tabata\opset.py:256: FutureWarning:

Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.

C:\Users\ruber\OneDrive\Bureau\Projet\tabata\opset.py:256: FutureWarning:

Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.

C:\Users\ruber\OneDrive\Bureau\Projet\tabata\opset.py:256: FutureWarning:

Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.

